# Sales Performance

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns

In [38]:
df = pd.read_csv("../data/sales_data.csv")
df.set_index(['Close Date']).head(2)

,Account Id,Account Name,Account Owned By,Account Type,Address,Billing City,Billing Region,Billing State,Closed,Created Date,...,Opportunity Name,Opportunity Type,Product Name,Stage,Won,Zip Code,Amount,Discount Granted,Opportunity Quantity,Unit Price
Close Date,,,,,,,,,,,,,,,,,,,,,
1/8/2011,001i0000008rFxEAAU,123 Warehousing,Tara Wang,Gold,"3270 N Hwy 87, Pine, AZ 85544",Pine,Southwest,Arizona,Vrai,1/8/2011,...,123 Warehousing vt8dAAA MOL Standard Maintenance,Maintenance,MOL Standard,Closed Lost,Faux,85544,196.8,18.00%,2.0,120
1/10/2011,001i0000008rFoZAAU,Mollis LLC,Tara Wang,Gold,"4746 E. Grant Rd., Tucson, AZ 85712",Tucson,Southwest,Arizona,Vrai,1/3/2011,...,Mollis LLC vt8VAAQ MOL Standard Software,Software,MOL Standard,Closed Lost,Faux,85712,21312.0,26.00%,36.0,800


In [39]:
df['year'] = pd.DatetimeIndex(df['Close Date']).year
df['month'] = pd.DatetimeIndex(df['Close Date']).month

In [40]:
df.head(2)

,Account Id,Account Name,Account Owned By,Account Type,Address,Billing City,Billing Region,Billing State,Close Date,Closed,...,Product Name,Stage,Won,Zip Code,Amount,Discount Granted,Opportunity Quantity,Unit Price,year,month
0,001i0000008rFxEAAU,123 Warehousing,Tara Wang,Gold,"3270 N Hwy 87, Pine, AZ 85544",Pine,Southwest,Arizona,1/8/2011,Vrai,...,MOL Standard,Closed Lost,Faux,85544,196.8,18.00%,2.0,120,2011,1
1,001i0000008rFoZAAU,Mollis LLC,Tara Wang,Gold,"4746 E. Grant Rd., Tucson, AZ 85712",Tucson,Southwest,Arizona,1/10/2011,Vrai,...,MOL Standard,Closed Lost,Faux,85712,21312.0,26.00%,36.0,800,2011,1


In [41]:
sales = df.groupby(['Billing Region'])['Amount'].agg('sum').reset_index(name='Total Sales ($)')
sales

,Billing Region,Total Sales ($)
0,Midwest,18490973.5
1,Northeast,19187054.2
2,Southeast,11157045.7
3,Southwest,5710699.7
4,West,23590369.0


In [42]:
perc_sales = df['Billing Region'].value_counts()
perc_sales

West         870
Northeast    861
Midwest      818
Southeast    406
Southwest    260
Name: Billing Region, dtype: int64

In [43]:
df['Close Date']= pd.to_datetime(df['Close Date'])

In [44]:
df['Close Date'].astype

<bound method NDFrame.astype of 0      2011-01-08
1      2011-01-10
2      2011-01-16
3      2011-01-19
4      2011-01-20
          ...    
3210   2015-08-19
3211   2015-08-20
3212   2015-08-21
3213   2015-08-23
3214   2015-08-29
Name: Close Date, Length: 3215, dtype: datetime64[ns]>

In [45]:
perc = df.groupby(pd.Grouper(key='Close Date',freq='Y')).sum()
perc

,Zip Code,Amount,Opportunity Quantity,Unit Price,year,month
Close Date,,,,,,
2011-12-31,7171317,2646313.2,4533.000000,130460,396167,1479
2012-12-31,12982746,7427210.9,11892.000000,260760,754500,2709
2013-12-31,32098311,15951462.3,28556.000000,505380,1497672,5382
2014-12-31,61633196,29029356.8,49072.000000,780820,2253666,7899
2015-12-31,51537255,23081798.9,40933.666667,540380,1571700,2944


In [46]:
sales_per = df.groupby(['year', 'month', 'Billing Region'])['Amount'].agg('sum').reset_index(name='Total Sales ($)')
sales_per

,year,month,Billing Region,Total Sales ($)
0,2011,1,Southwest,21508.8
1,2011,1,West,51268.0
2,2011,2,Southeast,63906.0
3,2011,2,Southwest,7104.0
4,2011,2,West,57344.2
...,...,...,...,...
248,2015,8,Midwest,322250.0
249,2015,8,Northeast,51462.0
250,2015,8,Southeast,4272.0
251,2015,8,Southwest,68016.0


# Chart 3

In [47]:
# Contribution

In [48]:
a = sales_per.groupby(['Billing Region'])['Total Sales ($)'].agg('sum')
a

Billing Region
Midwest      18490973.5
Northeast    19187054.2
Southeast    11157045.7
Southwest     5710699.7
West         23590369.0
Name: Total Sales ($), dtype: float64

In [49]:
b = sales_per.groupby(['Billing Region'])['Total Sales ($)'].agg('count')
b

Billing Region
Midwest      46
Northeast    54
Southeast    46
Southwest    53
West         54
Name: Total Sales ($), dtype: int64

In [50]:
b = sales_per.groupby(['year','Billing Region'])['Total Sales ($)'].agg('sum')
c = sales_per.groupby(['year'])['Total Sales ($)'].agg('sum')
z = b/c*100
z = z.reset_index(name='% Total Sales ($)')


# Product Performance

## Product Contribution

In [51]:
prod_grp = df.groupby(['Product Name'])['Amount'].agg('mean').reset_index(name='Total Sales ($)')
prod_grp['prod_perc'] = prod_grp['Total Sales ($)'].transform(lambda x: x / x.sum()*100)
prod_grp

,Product Name,Total Sales ($),prod_perc
0,MOL Mobile,22399.004457,28.860029
1,MOL Premium,31631.631965,40.755821
2,MOL Standard,23581.913718,30.384150


In [52]:
x = prod_grp['Product Name']
x.to_list()

['MOL Mobile', 'MOL Premium', 'MOL Standard']

In [53]:

labels = prod_grp['Product Name'].to_list()
labels

['MOL Mobile', 'MOL Premium', 'MOL Standard']

## Product Contribution and Total Sales

In [54]:
df.groupby(['Product Name'])['Amount'].agg('mean').reset_index(name='Avg Sales ($)')

,Product Name,Avg Sales ($)
0,MOL Mobile,22399.004457
1,MOL Premium,31631.631965
2,MOL Standard,23581.913718


## Product Performance by sector

In [55]:
df.columns

Index(['Account Id', 'Account Name', 'Account Owned By', 'Account Type',
       'Address', 'Billing City', 'Billing Region', 'Billing State',
       'Close Date', 'Closed', 'Created Date', 'Industry', 'Opportunity ID',
       'Opportunity Name', 'Opportunity Type', 'Product Name', 'Stage', 'Won',
       'Zip Code', 'Amount', 'Discount Granted', 'Opportunity Quantity',
       'Unit Price', 'year', 'month'],
      dtype='object')

In [56]:
df['Stage'].value_counts()

Closed Won     2120
Closed Lost     929
Discover         62
Commit           32
Decision         30
Solution         26
Qualify          16
Name: Stage, dtype: int64

In [57]:
stage = df.groupby(['Stage'])['Amount'].agg('sum').sort_values(ascending = False)
stage

Stage
Closed Won     44628054.6
Closed Lost    25211143.1
Discover        3192818.2
Qualify         1600501.0
Solution        1394951.6
Commit          1368271.8
Decision         740401.8
Name: Amount, dtype: float64

In [58]:

# df.groupby(['Product Name', 'Stage'])['Amount'].agg('sum').sort_values(ascending = False).reset_index(name='Sales ($)').sort_values('Product Name')


In [59]:
table = pd.pivot_table(df, values='Amount', index=['Product Name'],
                    columns=['Stage'], aggfunc=np.sum)
# table = table.drop(columns=['Commit', 'Decision', 'Discover', 'Qualify', 'Solution'])
table

Stage,Closed Lost,Closed Won,Commit,Decision,Discover,Qualify,Solution
Product Name,,,,,,,
MOL Mobile,1765408.8,5647829.4,118214.4,226925.4,121785.0,132246.0,28833.6
MOL Premium,2978180.5,5898277.0,701292.0,33672.0,227289.0,399876.0,547800.0
MOL Standard,20467553.8,33081948.2,548765.4,479804.4,2843744.2,1068379.0,818318.0


In [60]:
table2 = pd.pivot_table(df, values='Amount', index=['Stage'],
                    columns=['Product Name'], aggfunc=np.sum)
# table = table.drop(columns=['Commit', 'Decision', 'Discover', 'Qualify', 'Solution'])
table2

Product Name,MOL Mobile,MOL Premium,MOL Standard
Stage,,,
Closed Lost,1765408.8,2978180.5,20467553.8
Closed Won,5647829.4,5898277.0,33081948.2
Commit,118214.4,701292.0,548765.4
Decision,226925.4,33672.0,479804.4
Discover,121785.0,227289.0,2843744.2
Qualify,132246.0,399876.0,1068379.0
Solution,28833.6,547800.0,818318.0


In [61]:
table2.loc['Closed Lost':, 'MOL Mobile']

Stage
Closed Lost    1765408.8
Closed Won     5647829.4
Commit          118214.4
Decision        226925.4
Discover        121785.0
Qualify         132246.0
Solution         28833.6
Name: MOL Mobile, dtype: float64

In [62]:
table.columns.to_list()

['Closed Lost',
 'Closed Won',
 'Commit',
 'Decision',
 'Discover',
 'Qualify',
 'Solution']

In [97]:
table2 = pd.pivot_table(df, values='Amount', index=['Product Name', 'Stage'], aggfunc=np.sum).reset_index()
# table = table.drop(columns=['Commit', 'Decision', 'Discover', 'Qualify', 'Solution'])
table2

,Product Name,Stage,Amount
0,MOL Mobile,Closed Lost,1765408.8
1,MOL Mobile,Closed Won,5647829.4
2,MOL Mobile,Commit,118214.4
3,MOL Mobile,Decision,226925.4
4,MOL Mobile,Discover,121785.0
5,MOL Mobile,Qualify,132246.0
6,MOL Mobile,Solution,28833.6
7,MOL Premium,Closed Lost,2978180.5
8,MOL Premium,Closed Won,5898277.0
9,MOL Premium,Commit,701292.0


In [100]:
table2.iloc[0:2:]

,Product Name,Stage,Amount
0,MOL Mobile,Closed Lost,1765408.8
1,MOL Mobile,Closed Won,5647829.4


In [64]:
table2 = pd.pivot_table(df, values='Amount', index=['Stage'],
                    columns=['Product Name'], aggfunc=np.sum)
table2.iloc[:2]

Product Name,MOL Mobile,MOL Premium,MOL Standard
Stage,,,
Closed Lost,1765408.8,2978180.5,20467553.8
Closed Won,5647829.4,5898277.0,33081948.2


In [65]:
table.columns.to_list()

['Closed Lost',
 'Closed Won',
 'Commit',
 'Decision',
 'Discover',
 'Qualify',
 'Solution']

In [66]:
## map

In [85]:
map = df.groupby(['Billing State'])['Amount'].agg('sum').sort_values(ascending = False).reset_index()
map = map.set_index('Billing State')
map

,Amount
Billing State,
California,6725203.2
Pennsylvania,5776053.1
Michigan,5104674.3
New York,4893206.7
Oregon,4319599.2
Washington,4198121.4
Illinois,3756285.9
Arizona,3603453.5
North Carolina,3318520.8


In [80]:
df2 = pd.read_csv("../data/USA_Code.csv")
df2 = df2.set_index('State')
df2


,USPS Abbreviation,Traditional Abbreviation
State,,
Alabama,AL,Ala.
Alaska,AK,Alaska
Arizona,AZ,Ariz.
Arkansas,AR,Ark.
California,CA,Calif.
Colorado,CO,Colo.
Connecticut,CT,Conn.
Delaware,DE,Del.
Florida,FL,Fla.


In [89]:
result = pd.concat([map, df2], axis=1, join="inner")
result

,Amount,USPS Abbreviation,Traditional Abbreviation
California,6725203.2,CA,Calif.
Pennsylvania,5776053.1,PA,Pa.
Michigan,5104674.3,MI,Mich.
New York,4893206.7,NY,N.Y.
Oregon,4319599.2,OR,Ore. or Oreg.
Washington,4198121.4,WA,Wash.
Illinois,3756285.9,IL,Ill.
Arizona,3603453.5,AZ,Ariz.
North Carolina,3318520.8,NC,N.C.
Colorado,3151830.8,CO,Colo.


In [93]:
state = result.reset_index().drop(columns = ['Traditional Abbreviation'])
state

,index,Amount,USPS Abbreviation
0,California,6725203.2,CA
1,Pennsylvania,5776053.1,PA
2,Michigan,5104674.3,MI
3,New York,4893206.7,NY
4,Oregon,4319599.2,OR
5,Washington,4198121.4,WA
6,Illinois,3756285.9,IL
7,Arizona,3603453.5,AZ
8,North Carolina,3318520.8,NC
9,Colorado,3151830.8,CO


# Sun Brust OR Tree Map

In [95]:
sun_brust = df.groupby(['Billing Region','Industry', 'Product Name'])['Amount'].agg('sum').reset_index(name='Total Sales ($)')
sun_brust

,Billing Region,Industry,Product Name,Total Sales ($)
0,Midwest,Agriculture,MOL Standard,498885.4
1,Midwest,Appliances,MOL Standard,92752.0
2,Midwest,Automotives,MOL Premium,52236.0
3,Midwest,Automotives,MOL Standard,507353.0
4,Midwest,Business Services,MOL Premium,32410.0
...,...,...,...,...
258,West,Transportation,MOL Premium,456715.0
259,West,Transportation,MOL Standard,1948511.0
260,West,Utility,MOL Standard,34141.2
261,West,Wholesale,MOL Premium,8316.0


In [32]:
sun_brust['Product Name'].unique()

array(['MOL Mobile', 'MOL Premium', 'MOL Standard'], dtype=object)

# Top 10 Accounts

In [102]:
acc = df.groupby(['Billing Region','Account Owned By'])['Amount'].agg('sum').reset_index(name='Total Sales ($)')
top_10 = acc.sort_values(by= ['Total Sales ($)'], ascending = False)
top_10.head(10)

,Billing Region,Account Owned By,Total Sales ($)
16,West,Eric Barnes,8517720.6
17,West,Fred Jenkins,6725203.2
7,Northeast,Kevin Gibson,5776053.1
2,Midwest,Jesse Alexander,5229649.7
3,Midwest,Keyur Cruz,5104674.3
6,Northeast,Julie Owens,4893206.7
1,Midwest,Dennis Watson,4263974.1
18,West,Jennifer Gonzales,4252480.8
12,Southeast,Joe Ford,4232098.2
14,Southwest,Tara Wang,4121734.9


# Forecast


In [105]:
sales_by_month = df.groupby(['month', 'year'])['Amount'].agg('sum').reset_index(name='Total Sales ($)')
sales_by_month.head()

,month,year,Total Sales ($)
0,1,2011,72776.8
1,1,2012,732016.0
2,1,2013,633941.4
3,1,2014,1886067.6
4,1,2015,2572234.3


In [110]:
sales_2011  = sales_by_month[sales_by_month['year']==2011]
sales_2011.reset_index().drop(columns='index')

,month,year,Total Sales ($)
0,1,2011,72776.8
1,2,2011,128354.2
2,3,2011,212183.0
3,4,2011,19374.0
4,5,2011,96388.0
5,6,2011,324905.4
6,7,2011,165608.0
7,8,2011,159463.0
8,9,2011,398246.6
9,10,2011,159234.0
